In [1]:
from openml import tasks

import torch

import sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold

from classes import Dataset
from functions import train, eval, run_HPO_CV

In [2]:
oml_task_diabetes = tasks.get_task(37)

In [3]:
X, y, categorical_indicator, attribute_names = oml_task_diabetes.get_dataset().get_data()

In [4]:
X

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,tested_positive
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,tested_negative
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,tested_positive
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,tested_negative
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,tested_positive
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,tested_negative
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,tested_negative
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,tested_negative
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,tested_positive


In [5]:
Xy = X.copy()

In [6]:
tmp = Dataset(
    X=Xy.loc[:, Xy.columns != 'class'],
    y=Xy.loc[:, 'class'],
    class_pos='tested_positive'
)
len(tmp)
tmp[2]

(tensor([  8.0000, 183.0000,  64.0000,   0.0000,   0.0000,  23.3000,   0.6720,
          32.0000]),
 tensor(1.))

In [7]:
categorical_indicator

[False, False, False, False, False, False, False, False, True]

In [8]:
# outer
data_train_test, data_val = train_test_split(
    Xy,
    train_size=2/3,
    shuffle=True,
    stratify=Xy.loc[:, 'class']
)

# reset indices as StratifiedKFold assumes 0-(n-1) index
data_train_test = data_train_test.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)

# inner
cv_inner = StratifiedKFold(
    n_splits=5,
    shuffle=False  # TODO: set to True
)

'''
in each inner fold, run EAGGA + find an optimal pareto front of configurations for this specific fold
then in the outer fold (holdout), compare each fold-specific optimal pareto front + select optimum
'''
run_HPO_CV(cv_inner, data_train_test, epochs=10, batch_size=8)

running HPO: 3 total_layers, 3 nodes per hidden layer
fold 1 / 5
epoch 0 / 10, eval loss 2.1487443080315223
epoch 1 / 10, train loss 1.5183969994003956
epoch 1 / 10, eval loss 1.0990659663310418
epoch 2 / 10, train loss 0.9164551144035963
epoch 2 / 10, eval loss 0.8169549520199115
epoch 3 / 10, train loss 0.7826365914482337
epoch 3 / 10, eval loss 0.7497633558053237
epoch 4 / 10, train loss 0.7386656598402903
epoch 4 / 10, eval loss 0.7260314226150513
epoch 5 / 10, train loss 0.7164273519928639
epoch 5 / 10, eval loss 0.7163717058988718
epoch 6 / 10, train loss 0.7027919424267915
epoch 6 / 10, eval loss 0.7116998250667865
epoch 7 / 10, train loss 0.6941087796137884
epoch 7 / 10, eval loss 0.7098489587123578
epoch 8 / 10, train loss 0.6886570247320029
epoch 8 / 10, eval loss 0.7060251923707815
epoch 9 / 10, train loss 0.6844454992275971
epoch 9 / 10, eval loss 0.702165424823761
epoch 10 / 10, train loss 0.6804962375989327
epoch 10 / 10, eval loss 0.6988374224075904
fold 2 / 5
epoch 0 / 